In [1]:
# gbm이란 틀린 부분에 가중치를 더하면서 진행하는 알고리즘

# # 안건들여도 될것 같은거
#     boosting_type: str = 'gbdt',
#     learning_rate: float = 0.1, - 건들면 좋을 것 같긴한데 학습시간에 많은 영향이 있을 것으로 예상되어 마지막에
#     objective: Union[str, Callable, NoneType] = None,
#     class_weight: Union[Dict, str, NoneType] = None,
#     random_state: Union[int, numpy.random.mtrand.RandomState, NoneType] = None,
#     subsample: float = 1.0, - # 훈련 인스턴스의 하위 샘플 비율입니다
#     subsample_freq: int = 0, - # 서브샘플의 빈도, <=0은 활성화가 없음을 의미
#     n_jobs: int = -1, - # 병렬 스레드 수
#     silent: Union[bool, str] = 'warn', - # 부스팅을 실행하는 동안 메시지를 인쇄할지 여부
#     importance_type: str = 'split',

# # 100까지 올렸을 때 정확도가 올라가는 것으로 보임, 딱히 과적합이 양상이 보이지 않음
# # 120부터 4.2대로 나오기 시작했지만 150번째에서 4.19가 찍힘
#     num_leaves: int = 31, - 전체 Tree의 leave 수 이고, 디폴트값은 31
# 	- 개별 트리가 가질 수 있는 최대 리프의 개수를 지정, lightgbm복잡도를 제어하는 주 파라미터
# 	- 수가 커질 수록 모델의 깊이가 깊어지므로 모델 정확도가 높아지지만 과적합 발생 확률 또한 높아짐
# # 맥스딥스로는 큰 효과를 못봄, -1, 0, 11, 12, 에서 5.06, 5.07정도의 값이 나옴 15에서 5.02가 나오긴 함
#     max_depth: int = -1,  - tree의 최대 깊이, 모델 과적합을 다룰때 사용, 과적합으로 느끼면 값을 줄여야함

# # 과적합 개선에 사용되는 파라미터
#     reg_alpha: float = 0.0, - # 가중치에 대한 L1 정규화 항
#     reg_lambda: float = 0.0, - # 가중치에 대한 L2 정규화 항

# # 적합하도록 부스트된 나무의 수
#     n_estimators: int = 100,

# ------------------------------------------------------------------------------------------------------------------
# # bin구성을 위한 샘플 수
#     subsample_for_bin: int = 200000,

# # 트리의 리프 노드에서 추가 파티션을 만드는 데 필요한 최소 손실 감소
#     min_split_gain: float = 0.0,

# # 자식(리프)에 필요한 인스턴스 가중치(헤시안)의 최소 합계
#     min_child_weight: float = 0.001,

# # 자식(리프)에 필요한 최소 데이터
#     min_child_samples: int = 20,

# # 각 트리를 구성할 때 열의 하위 샘플 비율
#     colsample_bytree: float = 1.0,



In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import lightgbm as lgb

In [ ]:
# 데이터 로드

train = pd.read_parquet('./train.parquet')

In [ ]:
# 2만건으로 학습
# 1천건으로 테스트
from sklearn.utils import shuffle

# tr = shuffle(train.sort_values('base_date')[train['base_date'] < 20220701]).iloc[:20000]
# te = shuffle(train.sort_values('base_date')[train['base_date'] > 20220631]).iloc[:1000]


# 얼마나 균일하게 뽑히는지
tr = shuffle(train.sort_values('base_date')[train['base_date'] < 20220701])
te = shuffle(train.sort_values('base_date')[train['base_date'] > 20220631])


# 가이드를 기준으로 유의해서 상관계수 비교
# 가이드에 drop에서 빠진 컬럼도 상관계수 비교
# y_train = tr['target']
# X_train = tr.drop(['id','base_date', 'target','road_name', 'start_node_name', 'end_node_name','vehicle_restricted'], axis=1)

# y_test = te['target']
# X_test = te.drop(['id','base_date', 'target','road_name', 'start_node_name', 'end_node_name','vehicle_restricted'], axis=1)

y_train = tr['target']
X_train = tr.drop(['day_of_week', 'multi_linked', 'connect_code', 'height_restricted', 'id','base_date', 'target','road_name', 'start_node_name', 'end_node_name','vehicle_restricted'], axis=1)

y_test = te['target']
X_test = te.drop(['day_of_week', 'multi_linked', 'connect_code', 'height_restricted', 'id','base_date', 'target','road_name', 'start_node_name', 'end_node_name','vehicle_restricted'], axis=1)


print(X_train.shape)
print(y_train.shape)
print(X_test.shape)

In [ ]:
from sklearn.metrics import mean_absolute_error


mae_list = []
min_num = 10
min_stack = 0
stop_num = 3

# test, max_depth
# 맥스딥스로는 큰 효과를 못봄, -1, 0, 11, 12, 에서 5.06, 5.07정도의 값이 나옴 15에서 5.02가 나오긴 함
for i in range(-1, 20):
    LR = lgb.LGBMRegressor(random_state = 42,
                           max_depth = i
                          ).fit(X_train, y_train)

    pred = LR.predict(X_test)
    mae = mean_absolute_error(pred, y_test)
    
    if min_num > mae:
        min_stack = 0
        mae_list.append((i, mae))
        min_num = mae
        print('max_depth : ', i, ', mae : ', mae)
    elif min_stack != stop_num:
        min_stack += 1
        mae_list.append((i, mae))
        min_num = mae
        print('max_depth : ', i, ', mae : ', mae)
    elif min_stack == stop_num:
        break
            

In [ ]:
# test, num_leaves
# 100까지 올렸을 때 정확도가 올라가는 것으로 보임, 딱히 과적합이 양상이 보이지 않음
# 120부터 4.2대로 나오기 시작했지만 150번째에서 4.19가 찍힘
default = 100
max_num_t = 150

for i in range(default, max_num_t + 1):
    LR = lgb.LGBMRegressor(random_state = 42,
                           num_leaves  = i
                          ).fit(X_train, y_train)

    pred = LR.predict(X_test)
    mae = mean_absolute_error(pred, y_test)
    
    if min_num > mae:
        min_stack = 0
        mae_list.append((i, mae))
        min_num = mae
        print('num_leaves  : ', i, ', mae : ', mae)
    elif min_stack != stop_num:
        min_stack += 1
        mae_list.append((i, mae))
        min_num = mae
        print('num_leaves  : ', i, ', mae : ', mae)
    elif min_stack == stop_num:
        break